In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# scrapes fake and real news data from website from PolitiFact

In [5]:
import pandas as pd
import requests
import time
import urllib.request
from bs4 import BeautifulSoup

# create lists to store scrapped data

In [22]:
authors=[]
dates=[]
statements=[]
sources=[]
targets=[]

# create function to scrape the website

In [23]:
def scrape_website(page_number):
    page_num=str(page_number)
    URL='https://www.politifact.com/factchecks/list/?page='+page_num
    webpage=requests.get(URL)
    soup=BeautifulSoup(webpage.text,'html.parser')
    # get the location of information
    # location the author and the date
    statement_footer=soup.find_all('footer',attrs={'class':'m-statement__footer'})
    # location of the statement
    statement_quote=soup.find_all('div',attrs={'class':'m-statement__quote'})
    # location of the source
    statement_meta=soup.find_all('div',attrs={'class':'m-statement__meta'})
    # location of the target
    target=soup.find_all('div',attrs={'class':'m-statement__meter'})
    # loop through the statement footer
    for i in statement_footer:
        link1=i.text.strip()
        name_and_date=link1.split()
        first_name=name_and_date[1]
        last_name=name_and_date[2]
        full_name=first_name +' '+last_name
        month=name_and_date[4]
        day=name_and_date[5]
        year=name_and_date[6]
        date=month+' '+day+' '+year
        dates.append(date)
        authors.append(full_name)
    #loop through the statement_quote
    for i in statement_quote:
        link2=i.find_all('a')
        statement_text=link2[0].text.strip()
        statements.append(statement_text)
    # loop through the meta
    for i in statement_meta:
        link3=i.find_all('a')
        source_text=link3[0].text.strip()
        sources.append(source_text)
    # loop through the target
    for i in target:
        link4=i.find('div', attrs={'class':'c-image'}).find('img').get('alt')
        targets.append(link4)

# loop through webpages to scrape the data

In [24]:
n=2
for i in range(1, n):
    scrape_website(i)

# create the data frame

In [39]:
data=pd.DataFrame(columns=['author','statement','date','source','target'])
data['author']=authors
data['statement']=statements
data['date']=dates
data['source']=sources
data['target']=targets

# show the data

In [40]:
data.head()

,author,statement,date,source,target
0,Madison Czopek,U.S. public schools don't teach the Declaratio...,"April 7, 2023",Facebook posts,false
1,Paul Specht,“North Carolina ranks last in the country in K...,"April 7, 2023",North Carolina state House Democratic Caucus,barely-true
2,Jeff Cercone,"""Vitamins aren’t FDA approved, but Twinkies are.”","April 7, 2023",Facebook posts,barely-true
3,Loreben Tuquero,There have been no new variants since Dr. Anth...,"April 7, 2023",Instagram posts,false
4,Grace Abels,“Florida man charged with a hate crime” for de...,"April 7, 2023",Tweets,barely-true


# create a function to set a True for true news and anything else is False (0 & 1)

In [44]:
def get_target_number(text):
    if text=='true' or text=='mostly-true':
        return 1
    else:
        return 0

# create a function to set a True for true news and anything else is False (REAL & FAKE)

In [45]:
def get_target_label(text):
    if text=='true' or text=='mostly-true':
        return 'REAL'
    else:
        return 'FAKE'

# create new columns to store the data

In [46]:
data['target_number']=data['target'].apply(get_target_number)
data['target_label']=data['target'].apply(get_target_label)

In [47]:
data

,author,statement,date,source,target,target_number,target_label
0,Madison Czopek,U.S. public schools don't teach the Declaratio...,"April 7, 2023",Facebook posts,false,0,FAKE
1,Paul Specht,“North Carolina ranks last in the country in K...,"April 7, 2023",North Carolina state House Democratic Caucus,barely-true,0,FAKE
2,Jeff Cercone,"""Vitamins aren’t FDA approved, but Twinkies are.”","April 7, 2023",Facebook posts,barely-true,0,FAKE
3,Loreben Tuquero,There have been no new variants since Dr. Anth...,"April 7, 2023",Instagram posts,false,0,FAKE
4,Grace Abels,“Florida man charged with a hate crime” for de...,"April 7, 2023",Tweets,barely-true,0,FAKE
5,Sara Swann,Pouring soda on raw pork will cause parasites ...,"April 7, 2023",Facebook posts,pants-fire,0,FAKE
6,Gabrielle Settles,A video shows Florida Gov. Ron DeSantis saying...,"April 7, 2023",TikTok posts,pants-fire,0,FAKE
7,Sara Swann,A doctor tweeted that she doesn’t regret givin...,"April 6, 2023",Instagram posts,pants-fire,0,FAKE
8,Jeff Cercone,Report proves that COVID-19 vaccines caused 30...,"April 6, 2023",Instagram posts,false,0,FAKE
9,Madison Czopek,“President Trump is officially a political pri...,"April 6, 2023",Tom Fitton,false,0,FAKE


# save the data

In [49]:
data.to_csv('political_fact_checker.csv')